In [1]:
%pwd

'c:\\Users\\Aniruddh\\OneDrive\\Desktop\\ai-doc\\End-to-end-medical-chatbot\\research'

In [2]:

import os
os.chdir("../")

In [3]:

%pwd

'c:\\Users\\Aniruddh\\OneDrive\\Desktop\\ai-doc\\End-to-end-medical-chatbot'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data From the PDF File
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [6]:
extracted_data = load_pdf_file(
    data=r"C:\Users\Aniruddh\OneDrive\Desktop\ai-doc\End-to-end-medical-chatbot\Data"
)

In [7]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [8]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5859


In [9]:

from langchain.embeddings import HuggingFaceEmbeddings


In [10]:
def download_hugging_face_embeddings():
    embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [11]:
embeddings = download_hugging_face_embeddings()

C:\Users\Aniruddh\AppData\Local\Temp\ipykernel_15688\4043855499.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\Aniruddh\OneDrive\Desktop\ai-doc\End-to-end-medical-chatbot\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:

query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [16]:

from dotenv import load_dotenv
load_dotenv()

True

In [18]:

PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
groq_api_key=os.environ.get('groq_api_key')

In [19]:

from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medicalbot"


pc.create_index(
    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws", 
        region="us-east-1"
    ) 
)

{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-8cmigah.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}

In [20]:

import os
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY
os.environ["groq_api_key"] = groq_api_key


In [21]:
from langchain_pinecone import PineconeVectorStore

docsearch = PineconeVectorStore.from_documents(
    documents=text_chunks,
    index_name=index_name,
    embedding=embeddings, 
)

In [22]:
from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)

In [23]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [24]:
retrieved_docs = retriever.invoke("What is Acne?")

In [25]:
retrieved_docs

[Document(id='11211e4e-6774-43c4-987e-a0087fa7eb3a', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\Aniruddh\\OneDrive\\Desktop\\ai-doc\\End-to-end-medical-chatbot\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='dbefe0a7-eb5d-4017-b273-f35708307f06', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'C:\\Users\\Aniruddh\\OneDrive\\Desktop\\ai-doc\\End-to-end-medical-chatbot\\Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affectin

In [30]:
pip install groq


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: groq in c:\users\aniruddh\onedrive\desktop\ai-doc\end-to-end-medical-chatbot\medibot\lib\site-packages (0.31.0)



In [31]:
pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.Collecting langchain-groq
  Using cached langchain_groq-0.3.7-py3-none-any.whl.metadata (2.6 kB)
Using cached langchain_groq-0.3.7-py3-none-any.whl (16 kB)



In [43]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama3-70b-8192", 
    temperature=0.4,
    max_tokens=500,
    api_key= groq_api_key
)

In [44]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

In [45]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [46]:

response = rag_chain.invoke({"input": "what is acne"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back, occurring when the pores of the skin become clogged with oil, dead skin cells, and bacteria.


In [47]:

response = rag_chain.invoke({"input": "What is stats?"})
print(response["answer"])

I don't know what "stats" refers to in this context. It could be short for "statistics," but without more information, I can't provide a specific answer.
